In [73]:
import pandas as pd
import re
import collections
import numpy as np
train=pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
import re
temp = []
temp_test=[]
r=re.compile("[a-zA-Z0-9 ]")
for i in train.iloc[:,1]:
    i2=filter(r.match,i)
    temp.append(i2.lower())

for i in test.iloc[:,1]:
    i2=filter(r.match,i)
    temp_test.append(i2.lower())
    
bagofwords = [collections.Counter(re.findall(r'\w+',txt)) for txt in temp]
bagofwords_test=[collections.Counter(re.findall(r'\w+',txt)) for txt in temp_test]
print(bagofwords[1])

Counter({'me': 1, 'be': 1, 'that': 1, 'never': 1, 'it': 1, 'occurred': 1, 'to': 1, 'a': 1, 'fumbling': 1, 'the': 1, 'mistake': 1, 'might': 1, 'mere': 1, 'once': 1})


In [74]:
for i in range(len(bagofwords)):
    bagofwords[i] = dict(bagofwords[i])
for i in range(len(bagofwords_test)):
    bagofwords_test[i] = dict(bagofwords_test[i])

In [75]:
new_df = pd.DataFrame(bagofwords)
len(new_df.columns) 

25404

In [76]:
new_df2 = pd.DataFrame(bagofwords_test)
len(new_df2.columns)

17756

In [77]:
new_df = new_df.fillna(0)
new_df2 = new_df2.fillna(0)
new_df

,a,aaem,ab,aback,abaft,abandon,abandoned,abandoning,abandonment,abaout,...,zodiac,zodiacal,zoilus,zokkar,zone,zones,zopyrus,zorry,zubmizzion,zuro
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
#Removing words which have more than 5%  of words in the training dataset. We shall set the limit at words having frequency of more than 1000 are removed.
word_freq = new_df.sum(axis=0).order(ascending=False)
x = pd.DataFrame(word_freq[word_freq <= 1000])
extracted_words = list(x.index)

In [79]:
#Removing words which have more than 5% of words in the training dataset. We shall set the limit at words having frequency of more than 1000 are removed.
word_freq_test = new_df2.sum(axis=0).order(ascending=False)
x_test = pd.DataFrame(word_freq_test[word_freq_test <= 500])
extracted_words_test = list(x_test.index)

In [80]:
#Subsetting columns in training and testing dataset
new_df = new_df.loc[:,extracted_words]
new_df2 = new_df2.loc[:,extracted_words_test]

In [81]:
intersection = new_df.columns & new_df2.columns
intersection
joineddf = new_df2.loc[:,intersection]
other = [ x for x in new_df.columns if x not in intersection]
other
new_joined = pd.concat([joineddf,pd.DataFrame(columns=other)]).fillna(0)

In [82]:
from sklearn.naive_bayes import MultinomialNB
gnb = MultinomialNB()
pred = gnb.fit(new_df,train.iloc[:,2]).predict_proba(new_joined)

In [83]:
final = pd.concat([test.iloc[:,0],pd.DataFrame(pred,columns=['EAP','HPL','MWS'])],axis=1)

In [84]:
import csv
final.to_csv("output20percent.csv",float_format='%.20f',index=False)
